In [ ]:
pip install pycoingecko

from pycoingecko import CoinGeckoAPI
import time

cg = CoinGeckoAPI()

print("👋 Hey there! I’m CryptoBuddy – your guide to green and growing crypto investments!")

# Define crypto list to track
crypto_list = {
    "bitcoin": "Bitcoin",
    "ethereum": "Ethereum",
    "cardano": "Cardano"
}

# Sustainability scores (since API doesn’t provide this, we hardcode or estimate)
sustainability_scores = {
    "bitcoin": 3,
    "ethereum": 6,
    "cardano": 8
}

# Function to fetch live data from CoinGecko
def fetch_crypto_data():
    ids = ','.join(crypto_list.keys())
    market_data = cg.get_coins_markets(vs_currency='usd', ids=ids)
    
    data = {}
    for coin in market_data:
        name = coin['id']
        data[name] = {
            "current_price": coin['current_price'],
            "market_cap": coin['market_cap'],
            "market_cap_rank": coin['market_cap_rank'],
            "price_change_24h": coin['price_change_percentage_24h'],
            "sustainability_score": sustainability_scores.get(name, 5)
        }
    return data

# Decision logic based on live data
def crypto_advice_live(user_query, data):
    user_query = user_query.lower()

    if "sustainable" in user_query:
        recommend = max(data, key=lambda x: data[x]["sustainability_score"])
        return f"🌱 {crypto_list[recommend]} is your best bet – it's eco-friendly and future-focused!"

    elif "trending" in user_query or "rising" in user_query:
        trending = [crypto_list[k] for k, v in data.items() if v["price_change_24h"] and v["price_change_24h"] > 0]
        return f"📈 These cryptos are rising today: {', '.join(trending)}."

    elif "profitable" in user_query or "growth" in user_query:
        profitable = sorted(
            data.items(),
            key=lambda x: (x[1]["price_change_24h"] or 0) + x[1]["market_cap"],
            reverse=True
        )
        best = profitable[0][0]
        return f"💰 {crypto_list[best]} looks strong today with high growth potential!"

    elif "all data" in user_query:
        return '\n'.join([f"{crypto_list[k]}: {v}" for k, v in data.items()])

    else:
        return "❓ Not sure how to help. Try asking about trends, sustainability, or profitability!"

# Chat loop
while True:
    user_input = input("\nYou: ")
    if user_input.lower() in ["exit", "quit"]:
        print("CryptoBuddy: 🚀 Stay sharp and invest smart. See you next time!")
        break
    try:
        crypto_data = fetch_crypto_data()
        response = crypto_advice_live(user_input, crypto_data)
    except Exception as e:
        response = f"⚠️ Oops! Couldn’t fetch live data. Try again later. ({e})"
    print("CryptoBuddy:", response)
    time.sleep(1)  # Prevent API spamming
